In [1]:
# To keep the page organized, do all imports here
from sqlalchemy import create_engine
import pandas as pd
from scipy import stats

In [2]:
# Database credentials
postgres_user = 'dabc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
postgres_db = 'kickstarterprojects'

# Use the credentials to start a connection
engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(
    postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db))

In [3]:
projects_df = pd.read_sql_table('ksprojects', con=engine)

# Remove the connection
engine.dispose()

In [4]:
projects_df.shape

(65418, 13)

In [5]:
projects_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65418 entries, 0 to 65417
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   id             65418 non-null  int64         
 1   name           65418 non-null  object        
 2   category       65418 non-null  object        
 3   main_category  65418 non-null  object        
 4   deadline       65418 non-null  datetime64[ns]
 5   goal           65418 non-null  float64       
 6   launched       65418 non-null  datetime64[ns]
 7   state          65418 non-null  object        
 8   backers        65418 non-null  int64         
 9   country        65418 non-null  object        
 10  usd_pledged    64673 non-null  float64       
 11  currency       65418 non-null  object        
 12  pledged        65418 non-null  float64       
dtypes: datetime64[ns](2), float64(3), int64(2), object(6)
memory usage: 6.5+ MB


In [6]:
projects_df.head()

,id,name,category,main_category,deadline,goal,launched,state,backers,country,usd_pledged,currency,pledged
0,1147015301,"""All We Had"" Gets Into Cannes -- $10 or More G...",Documentary,Film & Video,2009-05-20 21:50:00,300.0,2009-04-30 22:10:30,failed,4,US,40.00,USD,40.00
1,1100844465,daily digest,Documentary,Film & Video,2009-06-01 02:20:00,700.0,2009-05-04 21:14:28,successful,14,US,700.00,USD,700.00
2,1316334968,Drive A Faster Car 2.0,Nonfiction,Publishing,2009-06-04 06:00:00,1000.0,2009-05-04 22:51:31,successful,32,US,1367.00,USD,1367.00
3,1304906577,Accidental to Edinburgh - PHASE 1: AIRFARE,Theater,Theater,2009-06-05 05:59:00,6000.0,2009-04-30 22:22:43,successful,24,US,6575.00,USD,6575.00
4,1099226462,Logical Guess Pictures' 2nd Horror Movie!,Film & Video,Film & Video,2009-06-06 00:45:00,500.0,2009-04-30 01:32:55,successful,22,US,501.66,USD,501.66


In [7]:
# Count the number of unique values in this column
projects_df['category'].nunique()

158

In [8]:
# Find the frequency of each value in the column
category_counts = projects_df['category'].value_counts()

# Only print the first 10, because 158 is too many to print
category_counts.head(10)

Product Design    3444
Documentary       2969
Music             2740
Shorts            2407
Tabletop Games    2208
Food              2126
Video Games       2011
Film & Video      1828
Fiction           1670
Fashion           1584
Name: category, dtype: int64

In [9]:
projects_df['main_category'].nunique()

15

In [10]:
main_category_counts = projects_df['main_category'].value_counts()

# There are only 15 main categories, so print them all.
main_category_counts

Film & Video    11666
Music            9395
Publishing       6901
Games            5739
Technology       5328
Art              4843
Design           4798
Food             4343
Fashion          3700
Theater          2015
Photography      1933
Comics           1791
Crafts           1474
Journalism        811
Dance             681
Name: main_category, dtype: int64

In [11]:
# How many countries?
projects_df['country'].nunique()

22

In [12]:
# How are the projects distributed over the countries?
country_counts = projects_df['country'].value_counts()
country_counts

US      52071
GB       5617
CA       2404
AU       1258
N,"0      744
DE        538
NL        457
FR        397
IT        374
ES        294
SE        267
NZ        236
DK        157
NO        120
IE         99
AT         98
BE         90
CH         88
MX         43
SG         29
HK         26
LU         11
Name: country, dtype: int64

In [13]:
projects_df['currency'].nunique()

13

In [14]:
currency_counts = projects_df['currency'].value_counts()
currency_counts

USD    52597
GBP     5722
CAD     2445
EUR     2399
AUD     1271
SEK      271
NZD      241
DKK      163
NOK      123
CHF       88
MXN       43
SGD       29
HKD       26
Name: currency, dtype: int64

In [15]:
projects_df['state'].nunique()

6

In [16]:
state_counts = projects_df['state'].value_counts()
state_counts

failed        33922
successful    23125
canceled       6427
live            938
undefined       695
suspended       311
Name: state, dtype: int64

In [17]:
null_rows = projects_df[projects_df.isnull().any(axis=1)]
null_rows.head(10)

,id,name,category,main_category,deadline,goal,launched,state,backers,country,usd_pledged,currency,pledged
9490,1189637889,"""Natural Born Hooker""",Theater,Theater,2012-04-24 08:59:00,25000.0,2012-03-14 19:05:44,undefined,0,"N,""0",NaN,USD,2621.00
15111,1244477824,Statpedia - The Collaborative Search Engine fo...,Software,Technology,2012-12-14 10:48:14,500.0,2012-11-14 10:48:14,undefined,0,"N,""0",NaN,USD,500.00
17484,1084993156,"Willy Tea, Chris Doud and Chums: The Children'...",Music,Music,2013-04-09 06:06:11,3750.0,2013-03-26 05:06:11,successful,0,"N,""0",NaN,USD,4593.00
17532,1379649454,"Juiette Z. Payne presents debut EP ""Eternal Da...",Music,Music,2013-04-11 15:50:25,5000.0,2013-03-02 15:50:25,successful,0,"N,""0",NaN,USD,5250.00
27334,1349042579,O'Films Production Launch!,Film & Video,Film & Video,2014-04-30 02:58:25,6000.0,2014-04-08 02:58:25,undefined,0,"N,""0",NaN,USD,6168.00
27443,1205956740,"Short Film - ""The Man Who Choked""",Film & Video,Film & Video,2014-05-02 12:47:00,1000.0,2014-04-16 13:21:21,undefined,0,"N,""0",NaN,GBP,1042.00
27522,1125500286,All About Amy,Film & Video,Film & Video,2014-05-04 17:41:13,3000.0,2014-04-04 17:41:13,undefined,0,"N,""0",NaN,USD,3225.00
27574,1292835145,WonderQuest ~ A Feature Length Music Film,Film & Video,Film & Video,2014-05-06 05:12:41,1111.0,2014-04-05 05:12:41,undefined,0,"N,""0",NaN,USD,3032.88
27602,1092574678,Pixelate Film Festival 2014,Film & Video,Film & Video,2014-05-07 13:37:16,1500.0,2014-04-09 13:37:16,undefined,0,"N,""0",NaN,GBP,1552.00
27755,1061706792,Zombie Shuffle feature film,Film & Video,Film & Video,2014-05-12 05:59:00,5000.0,2014-04-09 16:12:49,undefined,0,"N,""0",NaN,USD,5305.00


In [18]:
null_rows.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 745 entries, 9490 to 59365
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   id             745 non-null    int64         
 1   name           745 non-null    object        
 2   category       745 non-null    object        
 3   main_category  745 non-null    object        
 4   deadline       745 non-null    datetime64[ns]
 5   goal           745 non-null    float64       
 6   launched       745 non-null    datetime64[ns]
 7   state          745 non-null    object        
 8   backers        745 non-null    int64         
 9   country        745 non-null    object        
 10  usd_pledged    0 non-null      float64       
 11  currency       745 non-null    object        
 12  pledged        745 non-null    float64       
dtypes: datetime64[ns](2), float64(3), int64(2), object(6)
memory usage: 81.5+ KB


In [19]:
zero_filled = null_rows.fillna(0)
zero_filled.head(10)

,id,name,category,main_category,deadline,goal,launched,state,backers,country,usd_pledged,currency,pledged
9490,1189637889,"""Natural Born Hooker""",Theater,Theater,2012-04-24 08:59:00,25000.0,2012-03-14 19:05:44,undefined,0,"N,""0",0.0,USD,2621.00
15111,1244477824,Statpedia - The Collaborative Search Engine fo...,Software,Technology,2012-12-14 10:48:14,500.0,2012-11-14 10:48:14,undefined,0,"N,""0",0.0,USD,500.00
17484,1084993156,"Willy Tea, Chris Doud and Chums: The Children'...",Music,Music,2013-04-09 06:06:11,3750.0,2013-03-26 05:06:11,successful,0,"N,""0",0.0,USD,4593.00
17532,1379649454,"Juiette Z. Payne presents debut EP ""Eternal Da...",Music,Music,2013-04-11 15:50:25,5000.0,2013-03-02 15:50:25,successful,0,"N,""0",0.0,USD,5250.00
27334,1349042579,O'Films Production Launch!,Film & Video,Film & Video,2014-04-30 02:58:25,6000.0,2014-04-08 02:58:25,undefined,0,"N,""0",0.0,USD,6168.00
27443,1205956740,"Short Film - ""The Man Who Choked""",Film & Video,Film & Video,2014-05-02 12:47:00,1000.0,2014-04-16 13:21:21,undefined,0,"N,""0",0.0,GBP,1042.00
27522,1125500286,All About Amy,Film & Video,Film & Video,2014-05-04 17:41:13,3000.0,2014-04-04 17:41:13,undefined,0,"N,""0",0.0,USD,3225.00
27574,1292835145,WonderQuest ~ A Feature Length Music Film,Film & Video,Film & Video,2014-05-06 05:12:41,1111.0,2014-04-05 05:12:41,undefined,0,"N,""0",0.0,USD,3032.88
27602,1092574678,Pixelate Film Festival 2014,Film & Video,Film & Video,2014-05-07 13:37:16,1500.0,2014-04-09 13:37:16,undefined,0,"N,""0",0.0,GBP,1552.00
27755,1061706792,Zombie Shuffle feature film,Film & Video,Film & Video,2014-05-12 05:59:00,5000.0,2014-04-09 16:12:49,undefined,0,"N,""0",0.0,USD,5305.00


In [20]:
# Print the original size of the DataFrame
print(projects_df.size)

# Drop `NaN` rows
clean_projects = projects_df.dropna()

# Print the size of the modified DataFrame
print(clean_projects.size)

850434
840749


In [21]:
def series_statistics(column):
  print('statistics for column: {}'.format(column.name))
  print('The max value in the column: {}'.format(column.max()))
  print('The min value in the column: {}'.format(column.min()))
  print('The mode value in the column: {}'.format(column.mode()))
  print('The median value in the column: {}'.format(column.median()))
  print('The mean of the column: {}'.format(column.mean()))
  print('The std of the column: {}'.format(column.std()))

In [22]:
series_statistics(clean_projects['goal'])

statistics for column: goal
The max value in the column: 100000000.0
The min value in the column: 1.0
The mode value in the column: 0    5000.0
Name: goal, dtype: float64
The median value in the column: 5000.0
The mean of the column: 42094.83745674393
The std of the column: 1033205.2738987174


In [23]:
series_statistics(clean_projects['usd_pledged'])

statistics for column: usd_pledged
The max value in the column: 8596480.0
The min value in the column: 0.0
The mode value in the column: 0    0.0
Name: usd_pledged, dtype: float64
The median value in the column: 542.0
The mean of the column: 7900.242114883215
The std of the column: 72783.34386006395


In [24]:
clean_projects['goal'].describe()

count    6.467300e+04
mean     4.209484e+04
std      1.033205e+06
min      1.000000e+00
25%      2.000000e+03
50%      5.000000e+03
75%      1.500000e+04
max      1.000000e+08
Name: goal, dtype: float64

In [25]:
clean_projects['usd_pledged'].describe()

count    6.467300e+04
mean     7.900242e+03
std      7.278334e+04
min      0.000000e+00
25%      2.500000e+01
50%      5.420000e+02
75%      3.608380e+03
max      8.596480e+06
Name: usd_pledged, dtype: float64

In [26]:
clean_projects[['goal', 'usd_pledged', 'backers']].describe()

,goal,usd_pledged,backers
count,6.467300e+04,6.467300e+04,64673.000000
mean,4.209484e+04,7.900242e+03,104.336091
std,1.033205e+06,7.278334e+04,994.725411
min,1.000000e+00,0.000000e+00,0.000000
25%,2.000000e+03,2.500000e+01,2.000000
50%,5.000000e+03,5.420000e+02,13.000000
75%,1.500000e+04,3.608380e+03,56.000000
max,1.000000e+08,8.596480e+06,154926.000000


In [27]:
clean_projects['goal'].corr(clean_projects['usd_pledged'])

0.006423740131433381

In [28]:
clean_projects[['goal', 'usd_pledged', 'backers']].corr()

,goal,usd_pledged,backers
goal,1.000000,0.006424,0.003033
usd_pledged,0.006424,1.000000,0.569047
backers,0.003033,0.569047,1.000000


In [29]:
stats.pearsonr(clean_projects['backers'], clean_projects['usd_pledged'])

(0.5690472468189975, 0.0)

In [30]:
# Database credentials
postgres_user = 'dabc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
postgres_db = 'baseball'

# Use the credentials to start a connection
engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(
    postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db))

# SQL to select players and salaries
sql = 'SELECT p.playerid, birthyear, birthcountry, deathyear, namefirst, namelast, weight, height, bats, throws, yearid, teamid, lgid, salary FROM people p join salaries s on p.playerid = s.playerid'


# Use the query to create a DataFrame
players_df = pd.read_sql_query(sql, con=engine)

# Remove the connection
engine.dispose()

# Print two rows to see what you got
players_df.head(2)

,playerid,birthyear,birthcountry,deathyear,namefirst,namelast,weight,height,bats,throws,yearid,teamid,lgid,salary
0,barkele01,1955,USA,NaN,Len,Barker,225,77,R,R,1985,ATL,NL,870000
1,bedrost01,1957,USA,NaN,Steve,Bedrosian,200,75,R,R,1985,ATL,NL,550000


In [31]:
players_1990 = players_df.iloc[(players_df['yearid'] == 1990).values]
players_1990.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 867 entries, 3289 to 4155
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   playerid      867 non-null    object 
 1   birthyear     867 non-null    int64  
 2   birthcountry  867 non-null    object 
 3   deathyear     36 non-null     float64
 4   namefirst     867 non-null    object 
 5   namelast      867 non-null    object 
 6   weight        867 non-null    int64  
 7   height        867 non-null    int64  
 8   bats          867 non-null    object 
 9   throws        867 non-null    object 
 10  yearid        867 non-null    int64  
 11  teamid        867 non-null    object 
 12  lgid          867 non-null    object 
 13  salary        867 non-null    int64  
dtypes: float64(1), int64(5), object(8)
memory usage: 101.6+ KB


In [32]:
players_2000 = players_df.iloc[(players_df['yearid'] == 2000).values]
players_2000.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 836 entries, 12263 to 13098
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   playerid      836 non-null    object 
 1   birthyear     836 non-null    int64  
 2   birthcountry  836 non-null    object 
 3   deathyear     10 non-null     float64
 4   namefirst     836 non-null    object 
 5   namelast      836 non-null    object 
 6   weight        836 non-null    int64  
 7   height        836 non-null    int64  
 8   bats          836 non-null    object 
 9   throws        836 non-null    object 
 10  yearid        836 non-null    int64  
 11  teamid        836 non-null    object 
 12  lgid          836 non-null    object 
 13  salary        836 non-null    int64  
dtypes: float64(1), int64(5), object(8)
memory usage: 98.0+ KB


In [33]:
stats.ttest_ind(players_1990['salary'], players_2000['salary'])

Ttest_indResult(statistic=-16.90651573407297, pvalue=2.1593870550905857e-59)

In [34]:
sample_1_n = players_1990.shape[0]
sample_2_n = players_2000.shape[0]
sample_1_mean = players_1990['salary'].mean()
sample_2_mean = players_2000['salary'].mean()
sample_1_var = players_1990['salary'].var()
sample_2_var = players_2000['salary'].var()

In [36]:
import math
std_err_difference = math.sqrt((sample_1_var/sample_1_n)+(sample_2_var/sample_2_n))

mean_difference = sample_2_mean - sample_1_mean

margin_of_error = 1.96 * std_err_difference
ci_lower = mean_difference - margin_of_error
ci_upper = mean_difference + margin_of_error

print("The difference in means at the 95% confidence interval is between "+str(ci_lower)+" and "+str(ci_upper)+".")

The difference in means at the 95% confidence interval is between 1306451.096390415 and 1655570.6584535995.


In [37]:
def get_95_ci(array_1, array_2):
    sample_1_n = array_1.shape[0]
    sample_2_n = array_2.shape[0]
    sample_1_mean = array_1.mean()
    sample_2_mean = array_2.mean()
    sample_1_var = array_1.var()
    sample_2_var = array_2.var()
    mean_difference = sample_2_mean - sample_1_mean
    std_err_difference = math.sqrt((sample_1_var/sample_1_n)+(sample_2_var/sample_2_n))
    margin_of_error = 1.96 * std_err_difference
    ci_lower = mean_difference - margin_of_error
    ci_upper = mean_difference + margin_of_error
    return("The difference in means at the 95% confidence interval (two-tail) is between "+str(ci_lower)+" and "+str(ci_upper)+".")

get_95_ci(players_1990['salary'],players_2000['salary'])

'The difference in means at the 95% confidence interval (two-tail) is between 1306451.096390415 and 1655570.6584535995.'